### CS152 Assingment 1
#### Raymundo Gonzalez Leal

Here is a general solution to the 8-star puzzle for any acceptable number of tiles. I assume that the goal solution is ascending order (so [[0,1,2],[3,4,5],[6,7,8]] for the 8-puzzle case). Hence our program will return an error code if it is not possible to achieve this solution from a given initial configuration.

In [90]:
#A generalization of the 8-star puzzle solved through A* search
from queue import PriorityQueue

class PuzzleNode:
    

    def __init__(self, state, f, g, parent=None):
        """
        State is a list of lists defining the whole puzzle
        f and g will be used to store costs for A*
        the parent pointer can be used to reconstruct optimal path
        """
        self.state = state
        self.f = f
        self.g = g
        self.parent = parent
        self.pruned = False
        
    def coord_blank(self):
        #find coordinates of blank (0) tile
        #the solver will use this to add states to the frontier
        for y in range(len(self.state)):
            for x in range(len(self.state[0])):
                if self.state[y][x] == 0: return y, x    

    
    # Based on the 2 disjoint sets explanation 
    #from: http://www.8puzzle.com/8_puzzle_algorithm.html
    def solvable(self):
        Sum = 0
        flat_state = [y for x in self.state for y in x]
        for i in range(len(flat_state)):
            val = flat_state[i]
            for other_val in flat_state[i:]:            
                if other_val < val and other_val > 0 : Sum +=1
        return Sum % 2 == 0
        
    
    # Print grid
    def __str__(self):
        return "\n".join([" ".join([str(x) for x in row]) 
                          for row in self.state])
        

def heuristic_uniform(state): return 0

## Count number of misplaced tiles
def misplaced_tiles(state):
    n = len(state)
    val = 0
    for row_ind in range(n):
        for col_ind in range(n):
            if state[row_ind][col_ind] != row_ind*n + col_ind: val+=1
    return val

#Manhattan distance for every tile
#On every move we move two tiles. In the best case scenario, 
#this move gets both tiles closer to their destination.
#Hence, we will take the manhattan distance for each tile, 
#and divide it by 2.
def manhattan_distance(state, goal = None):
    n = len(state)
    dist = 0    
    for row_ind, row in enumerate(state):
        for col_ind, item in enumerate(row):            
            row_ind_goal = int(item/n)
            col_ind_goal = item%n
            dist += abs(row_ind_goal - row_ind) 
            + abs(col_ind_goal - col_ind)
    return dist / 2.0


    
heuristics = [misplaced_tiles, manhattan_distance]

In [64]:
def solvePuzzle(n, state, heuristic, output = False):
    """
    INPUT ARGUMENTS
    —n - the puzzle dimension 
    —state - the starting state
    —heuristic - a handle to a heuristic function
    —prnt - whether or not to print the solution
    OUTPUT ARGUMENTS
    —steps - the number of steps required to reach the goal 
    —frontierSize - maximum size of the frontier during search
    —err - an error code (-1 for bad input size or numbers)
    """
    
    #Check dimensions and numbers from 0 to n exactly once
    #For duplicates, we can simply check if every number 
    #from 0 to n appears (if they are appear and size is right,
    #then there are no duplicates). 
    
    if (len(state) != n or not all([len(state[i]) == n 
                                    for i in range(n)])  
        or not all([any(i in row for row in state) 
                    for i in range(n)])): return 0,0,-1

    # Create start node
    start_node = PuzzleNode(state, heuristic(state), 0)
    
    if not start_node.solvable():
        return 0, 0, -2

    goal = [[row_ind*n + col_ind for col_ind in range(n)] 
            for row_ind in range(n)]
    
    # Dictionary with current cost to reach all visited nodes
    # can't use list as key so we turn it to str
    costs_db = {str(state): start_node}

    frontier = PriorityQueue()
    frontier.put(start_node)

    # Begin A* Search
    expansion_counter = 0

    while not frontier.empty():
        # Get front of frontier (priority)
        cur_node = frontier.get()

        if cur_node.pruned:
            continue # Skip if this node has been marked for removal

        # Check if we are at the goal
        if cur_node.state == goal: break
        
        # Get coordinates of blank tile for future checks
        y_0, x_0 = cur_node.coord_blank()

        # Check all 4 options for moving blank tile
        for dx, dy in [(-1, 0), (1,0), (0,1), (0,-1)]:

            if y_0 + dy in range(0, n) and x_0 + dx in range(0, n):
               
                # swap blank space in copy of state
                next_state = [row[:] for row in cur_node.state]               
                next_state[y_0][x_0] = cur_node.state[y_0 + dy][x_0 + dx]
                next_state[y_0 + dy][x_0 + dx] = 0
                g = cur_node.g + 1 

                # If we explored same grid layout before
                if str(next_state) in costs_db:
                    if costs_db[str(next_state)].g > g:
                        # Previous path to node is worse
                        costs_db[str(next_state)].pruned = True 
                    else:
                        continue # New path is worse

                h = heuristic(next_state) 
                # Add best cost to new node and put it in frontier
                next_node = PuzzleNode(next_state, g+h, g, cur_node)
                frontier.put(next_node)
                costs_db[str(next_state)] = next_node 

        expansion_counter = expansion_counter + 1 # We finished expanding the node

    if output:
        # Reconstruct the optimal path using parent pointers
        optimal_path = [cur_node.state]
        while cur_node.parent:
            optimal_path.append((cur_node.parent).state)
            cur_node = cur_node.parent
        print("A* search completed in %d steps\n"% expansion_counter)
        print("Optimal Path to Goal:\n")
        for grid in optimal_path[::-1]:
            print(grid)
        print("Current frontier size: %d"%frontier.qsize())
    
    return expansion_counter, frontier.qsize(), 0

In [89]:
import time

start = time.time()
for state in [[[5,7,6],[2,4,3],[8,1,0]], [[7,0,8],[4,6,1],[5,3,2]], 
              [[2,3,7],[1,8,0],[6,5,4]]]:
    print(state)
    for heuristic in heuristics:
        print(solvePuzzle(len(state[0]), state, heuristic, False))
print(time.time() - start)

[[5, 7, 6], [2, 4, 3], [8, 1, 0]]
(55440, 28562, 0)
(131814, 55977, 0)
[[7, 0, 8], [4, 6, 1], [5, 3, 2]]
(26118, 16294, 0)
(31954, 19519, 0)
[[2, 3, 7], [1, 8, 0], [6, 5, 4]]
(281962, 68295, 0)
(299365, 72293, 0)
115.874067068


Due to runtime considerations I did not print the results from the uniform heuristic here (As it was the slowest). We see that Manhattan distance outperforms number of misplaced tiles (which performs uniform). This makes sense as it is the least optimistic of the heuristics (while still guaranteeing that the heuristic costs is always smaller than the actual cost to get to the goal from there). 

### Extension 1

I used the explanation from: http://www.8puzzle.com/8_puzzle_algorithm.html
My solvable function in the PuzzleNode class counts how many elements after a tile have a value smaller than that tile, and adds that value for every tile (explanation clearer in the link). We return true if said sum is even.